<a href="https://colab.research.google.com/github/olinyoder2534/TensorflowPractice/blob/main/ProductReview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gensim
import pandas as pd

In [2]:
import matplotlib.pyplot as plt

In [3]:
pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 9.3 MB/s eta 0:00:00


# Exploration

In [4]:
#dataset contains reviews of phone accessories
df = pd.read_json("/content/reviews_Cell_Phones_and_Accessories_5.json.gz", lines=True)
#df

In [5]:
df.head(4)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"


In [6]:
print("Max: {}".format(df['reviewText'].str.len().max()))
print("Mean: {}".format(round(df['reviewText'].str.len().mean(),3)))
print("Median: {}".format(df['reviewText'].str.len().median()))

Max: 32110
Mean: 491.84
Median: 248.0


In [7]:
df['overall'].value_counts()
#mostly positive reviews

overall
5    108664
4     39993
3     21439
1     13279
2     11064
Name: count, dtype: int64

In [8]:
print("Mean: {}".format(round(df['overall'].mean(),3)))
print("Median: {}".format(df['overall'].median()))

Mean: 4.13
Median: 5.0


In [9]:
df.shape

(194439, 9)

In [10]:
print(df.dtypes)

reviewerID        object
asin              object
reviewerName      object
helpful           object
reviewText        object
overall            int64
summary           object
unixReviewTime     int64
reviewTime        object
dtype: object


In [11]:
df.isna().sum()
#not going to need reviewName so fine to leave NAs in

reviewerID           0
asin                 0
reviewerName      3519
helpful              0
reviewText           0
overall              0
summary              0
unixReviewTime       0
reviewTime           0
dtype: int64

# Preprocessing

In [12]:
#I don't have enough RAM to run models using the entire dataset, so I'm taking a random subset of 10,000 records

subset_df = df.sample(n=10000, random_state=42)

In [13]:
subset_df.shape

(10000, 9)

In [14]:
subset_df.head(4)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
156187,A1P3ACSVXZ1JUM,B00C0ZKI9Q,LO127,"[0, 0]",iBOLT xProDock Active Car Dock/Holder/Mount fo...,5,Fits perfectly,1372032000,"06 24, 2013"
102252,A2Z3IZDQUIDRR9,B008FQVO5G,yogitim,"[8, 8]",This pouch is everything that I was looking fo...,5,Great Pouch!,1361577600,"02 23, 2013"
23146,A3EOVXI1VZIHUQ,B003Y74AZ2,"Z. Freeman ""Zach""","[1, 2]",First of all - the case I had on my iPhone 4 p...,3,An average iPhone case - nothing fancy,1296604800,"02 2, 2011"
86461,AWU526GBZ64MH,B007ZL0TJ4,Bree,"[0, 0]",I ordered this case because I had the same cas...,5,Great Case,1379635200,"09 20, 2013"


In [15]:
review_text = subset_df.reviewText.apply(gensim.utils.simple_preprocess)
review_text

156187    [ibolt, xprodock, active, car, dock, holder, m...
102252    [this, pouch, is, everything, that, was, looki...
23146     [first, of, all, the, case, had, on, my, iphon...
86461     [ordered, this, case, because, had, the, same,...
62407     [bought, few, of, these, as, gifts, they, were...
                                ...                        
121486    [bought, this, specific, adapter, because, the...
307       [ve, been, buying, these, headsets, for, sever...
80726     [the, tylt, powerplant, is, essentially, recha...
40551     [paid, less, than, for, this, product, for, my...
66990     [was, hesitant, buying, this, as, have, never,...
Name: reviewText, Length: 10000, dtype: object

In [16]:
review_text.reset_index(drop=True, inplace=True)
review_text.loc[0]

['ibolt',
 'xprodock',
 'active',
 'car',
 'dock',
 'holder',
 'mount',
 'for',
 'samsung',
 'galaxy',
 'note',
 'fits',
 'my',
 'galaxy',
 'perfectly',
 'had',
 'to',
 'reverse',
 'where',
 'the',
 'charging',
 'cable',
 'went',
 'so',
 'that',
 'was',
 'not',
 'hiding',
 'the',
 'volume',
 'button',
 'but',
 'other',
 'then',
 'that',
 'it',
 'was',
 'good',
 'it',
 'is',
 'very',
 'sturdy',
 'and',
 'works',
 'as',
 'good',
 'as',
 'the',
 'motorola',
 'nav',
 'dock',
 'had',
 'for',
 'my',
 'droid',
 'razr',
 'the',
 'view',
 'from',
 'the',
 'camera',
 'is',
 'unobstructed',
 'the',
 'software',
 'that',
 'you',
 'down',
 'load',
 'works',
 'ok',
 'was',
 'used',
 'to',
 'motorola',
 'drive',
 'app',
 'but',
 'this',
 'has',
 'just',
 'as',
 'much',
 'functionality',
 'you',
 'can',
 'program',
 'it',
 'to',
 'toggle',
 'bluetooth',
 'and',
 'all',
 'that']

In [17]:
review_text

0       [ibolt, xprodock, active, car, dock, holder, m...
1       [this, pouch, is, everything, that, was, looki...
2       [first, of, all, the, case, had, on, my, iphon...
3       [ordered, this, case, because, had, the, same,...
4       [bought, few, of, these, as, gifts, they, were...
                              ...                        
9995    [bought, this, specific, adapter, because, the...
9996    [ve, been, buying, these, headsets, for, sever...
9997    [the, tylt, powerplant, is, essentially, recha...
9998    [paid, less, than, for, this, product, for, my...
9999    [was, hesitant, buying, this, as, have, never,...
Name: reviewText, Length: 10000, dtype: object

## Word2Vec

In [18]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4,
)

In [19]:
model.build_vocab(review_text, progress_per=1000)

In [20]:
model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)

(3194603, 4400430)

In [21]:
model.wv.most_similar("good")

[('great', 0.7572157382965088),
 ('decent', 0.7571279406547546),
 ('ok', 0.6827141046524048),
 ('reasonable', 0.6689854264259338),
 ('awesome', 0.6335814595222473),
 ('fantastic', 0.6319836378097534),
 ('cool', 0.6208630204200745),
 ('excellent', 0.604057252407074),
 ('okay', 0.6010156869888306),
 ('nice', 0.5965790748596191)]

In [22]:
model.wv.most_similar("bad")

[('guess', 0.7621824145317078),
 ('okay', 0.7313181161880493),
 ('ok', 0.7226905226707458),
 ('pricy', 0.7054151296615601),
 ('rave', 0.6704893112182617),
 ('expect', 0.6527695655822754),
 ('expecting', 0.6524638533592224),
 ('fault', 0.6520946025848389),
 ('cheap', 0.6511734127998352),
 ('complain', 0.648902952671051)]

In [23]:
model.wv.most_similar("okay")

[('ok', 0.8932271599769592),
 ('alright', 0.8159366250038147),
 ('yes', 0.7633716464042664),
 ('pleasing', 0.7576489448547363),
 ('cheaply', 0.7566352486610413),
 ('plain', 0.7554841041564941),
 ('pic', 0.7549519538879395),
 ('tho', 0.7544816136360168),
 ('upset', 0.752570629119873),
 ('understated', 0.7515147924423218)]

# Sentiment Analysis

Modeling

In [24]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

In [25]:
#convert text to vector
def text_to_vector(text):
    words = text.split()
    vector = np.zeros((model.vector_size,))
    for word in words:
        if word in model.wv:
            vector += model.wv[word]
    return vector

In [26]:
X = np.array([text_to_vector(text) for text in subset_df['reviewText']])
y = subset_df['overall']

In [27]:
X[0]

array([ 2.33777768e+01,  2.16612013e+01, -5.81815609e+00,  3.03432441e+01,
        4.74539017e+00, -5.16197546e+01,  1.56270382e+01,  2.54178442e+01,
        1.70493310e+00, -3.04724149e+01, -2.19924174e+01, -2.85874600e+01,
       -6.61200829e+00,  1.50092818e+01,  2.49151092e+01,  8.27593286e+00,
        1.54228321e+01, -2.78583917e+01, -2.73055339e+01, -2.66160286e+01,
       -5.42779369e+00,  4.84179578e+00,  1.93623699e+01,  8.17645009e-01,
        2.36307241e+00,  2.13210215e+01, -1.71099563e+01,  1.16749534e+01,
        1.91650820e+00,  1.44133877e-01, -5.05134274e+00, -1.05414935e+01,
        1.71010455e+01, -2.43469173e+00, -1.77073344e+01,  2.40245759e+01,
        1.48006652e+01, -9.01339022e+00, -1.22147501e+01, -4.62504660e+01,
       -1.99479775e+01,  1.00931558e+01, -5.32940099e-01, -2.02986685e+00,
        2.37915635e+01,  1.38915949e+01, -1.31918470e+01, -7.91440691e-01,
        2.32330200e+01,  1.46813821e+01,  5.76238557e+00, -6.12856671e+01,
       -1.59565775e+01, -

In [28]:
y

156187    5
102252    5
23146     3
86461     5
62407     5
         ..
121486    5
307       5
80726     4
40551     5
66990     5
Name: overall, Length: 10000, dtype: int64

In [29]:
y.reset_index(drop=True, inplace=True)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Naive Bayes

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [32]:
#make vectors only contain values that are non-negative
def preprocess_embeddings(embeddings):
    min_embedding = np.min(embeddings)
    if min_embedding < 0:
        embeddings -= min_embedding
    return embeddings

In [33]:
X_train_preprocessed = preprocess_embeddings(X_train)
X_test_preprocessed = preprocess_embeddings(X_test)

In [34]:
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_preprocessed, y_train)

MultinomialNB()

In [35]:
y_predNB = naive_bayes.predict(X_test_preprocessed)

accuracyNB = accuracy_score(y_test, y_predNB)
accuracyNB

0.5186666666666667

In [36]:
y_predNB[:10]

array([5, 5, 5, 4, 5, 5, 5, 5, 4, 5])

In [37]:
y_test[:10]

6252    5
4684    5
1731    3
4742    5
4521    5
6340    5
576     4
5202    3
6363    1
439     5
Name: overall, dtype: int64

In [38]:
#tolerance of 1 (e.g. a prediction of 4 when the real value was 5 will be marked correct)
correct_predictions = 0

for pred, true in zip(y_predNB, y_test):
    if abs(pred - true) <= 1:
        correct_predictions += 1

accuracyNB2 = correct_predictions / len(y_test)
accuracyNB2

0.8016666666666666

## Random Forest

In [39]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [40]:
rf_classifier = RandomForestClassifier()

#grid search hyperparameters
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_
print("Best Parameters:", best_params)
print("Best Score:", best_score)

Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV] END max_depth=None, min_samples_split=2, n_estimators=100; total time=   9.5s
[CV] END max_depth=None, min_samples_split=2, n_estimators=100; total time=   5.2s
[CV] END max_depth=None, min_samples_split=2, n_estimators=100; total time=   5.9s
[CV] END max_depth=None, min_samples_split=2, n_estimators=200; total time=  11.0s
[CV] END max_depth=None, min_samples_split=2, n_estimators=200; total time=  11.0s
[CV] END max_depth=None, min_samples_split=2, n_estimators=200; total time=  10.3s
[CV] END max_depth=None, min_samples_split=2, n_estimators=300; total time=  16.5s
[CV] END max_depth=None, min_samples_split=2, n_estimators=300; total time=  15.9s
[CV] END max_depth=None, min_samples_split=2, n_estimators=300; total time=  16.5s
[CV] END max_depth=None, min_samples_split=5, n_estimators=100; total time=   5.9s
[CV] END max_depth=None, min_samples_split=5, n_estimators=100; total time=   5.0s
[CV] END max_depth=None, m

In [41]:
best_rf_classifier = grid_search.best_estimator_
y_predRF = best_rf_classifier.predict(X_test)

In [42]:
accuracyRF = accuracy_score(y_test, y_predRF)
accuracyRF
#barely better than random guessing (1/5 or .2)

0.21066666666666667

In [43]:
#tolerance of 1 (e.g. a prediction of 4 when the real value was 5 will be marked correct)
correct_predictionsRF = 0

for pred, true in zip(y_predRF, y_test):
    if abs(pred - true) <= 1:
        correct_predictionsRF += 1

accuracyRF2 = correct_predictionsRF / len(y_test)
accuracyRF2
#RF is often 1 off from the true value (if it were high-medium-low instead of 1-5, rf would do quite well)

0.8806666666666667

## RNN using LSTM

In [ ]:
max_length = subset_df['reviewText'].str.len().max()
X_train = pad_sequences(X_train, maxlen=max_length)
X_test = pad_sequences(X_test, maxlen=max_length)

In [ ]:
X_train.dtype

dtype('int32')

In [ ]:
subset_df['reviewText'].dtypes

dtype('O')

In [ ]:
vocab_size = len(model.wv.index_to_key)
embedding_dim = model.vector_size
num_classes = len(subset_df['overall'].unique())

In [ ]:
modelLSTM = Sequential()
modelLSTM.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
modelLSTM.add(LSTM(units=100))
modelLSTM.add(Dense(units=num_classes, activation='softmax'))

In [ ]:
y_train_adjusted = y_train - 1
y_test_adjusted = y_test - 1

In [ ]:
modelLSTM.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
modelLSTM.fit(X_train, y_train_adjusted, epochs=5, batch_size=32, validation_data=(X_test, y_test_adjusted))

Epoch 1/5
 11/219 [>.............................] - ETA: 7:37:04 - loss: 1.5533 - accuracy: 0.5114

In [ ]:
modelLSTM.evaluate(X_test,y_test)

In [ ]:
predictionsLSTM = modelLSTM.predict(X_test) + 1
np.argmax(predictionsLSTM[1])

In [ ]:
y_test[1]

# Auto Complete

In [48]:
# Convert text data into numerical sequences using Word2Vec embeddings
def text_to_sequence(text, model):
    words = text.split()
    sequence = [model.key_to_index[word] for word in words if word in model.key_to_index]
    return sequence

In [51]:
from keras.preprocessing.sequence import pad_sequences

max_sequence_length = subset_df['reviewText'].str.len().max()

X_sequences = [text_to_sequence(text, model.wv) for text in subset_df['reviewText']]
X_padded = pad_sequences(X_sequences, maxlen=max_sequence_length, padding='post')

X_sequences = np.array(X_padded)
X_sequences

array([[   7,  113,    8, ...,    0,    0,    0],
       [ 921,    4,  334, ...,    0,    0,    0],
       [   6,   36,    0, ...,    0,    0,    0],
       ...,
       [   4, 2241, 2408, ...,    0,    0,    0],
       [ 694,  258,   56, ...,    0,    0,    0],
       [  19, 2519,  489, ...,    0,    0,    0]], dtype=int32)

In [59]:
X_sequences.shape

(10000, 25435)

In [60]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_sequences, y, test_size=0.3, random_state=42)

In [54]:
vocab_size = len(model.wv.index_to_key)
embedding_dim = model.vector_size
#num_classes = len(subset_df['overall'].unique())

In [55]:
model_rnn = Sequential()
model_rnn.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
model_rnn.add(LSTM(units=100, return_sequences=True))
model_rnn.add(LSTM(units=100)
model_rnn.add(Dense(vocab_size, activation='softmax'))

In [56]:
model_rnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [57]:
print(model_rnn.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25435, 100)        994000    
                                                                 
 lstm (LSTM)                 (None, 25435, 100)        80400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 9940)              1003940   
                                                                 
Total params: 2158740 (8.23 MB)
Trainable params: 2158740 (8.23 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
#not enough RAM :(, might use a HMM instead
history = model_rnn.fit(X_train1, y_train1, epochs=5, batch_size=32, validation_split=0.1)

Epoch 1/10
